In [16]:
import pandas as pd
import pyodbc
import sqlite3

# iedereen heeft andere servername btw

In [17]:
servername = 'LAPTOP-C1FMPSTV\\SQLEXPRESS01'

# Northwind connectie

In [18]:
northwind_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=NorthWind; Trusted_Connection=yes')
nwCursor = northwind_conn.cursor()
nwCursor

# Adventureworks connectie

In [19]:
adventureworks_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=AdventureWorks2019; Trusted_Connection=yes')
awCursor = adventureworks_conn.cursor()
awCursor

# Aenc connectie

In [20]:
aenc_conn = pyodbc.connect('DRIVER={Microsoft Access Driver (*.mdb, *.accdb)}; DBQ=../raw-databases/aenc.accdb')
aencCursor = aenc_conn.cursor()

table_names = [table_info.table_name for table_info in aencCursor.tables()]

# Print table names
print(table_names)

['MSysAccessStorage', 'MSysACEs', 'MSysComplexColumns', 'MSysIMEXColumns', 'MSysIMEXSpecs', 'MSysNameMap', 'MSysNavPaneGroupCategories', 'MSysNavPaneGroups', 'MSysNavPaneGroupToObjects', 'MSysNavPaneObjectIDs', 'MSysObjects', 'MSysQueries', 'MSysRelationships', 'MSysResources', 'bonus', 'customer', 'department', 'employee', 'product', 'region', 'sales_order', 'sales_order_item', 'state']


In [21]:
aenctest = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
awtest = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
nwtest = pd.read_sql_query('SELECT * FROM customers', northwind_conn)

# print(aenctest)
# print(awtest)
# print(nwtest)

C:\Users\sheng\AppData\Local\Temp\ipykernel_41056\825424140.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aenctest = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_41056\825424140.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  awtest = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_41056\825424140.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nwtest = pd.read_sql_query('SELECT * FROM custo

AdventureWorks Person.Person / Person.Address / Person.PersonPhone

In [22]:
df_person = pd.read_sql_query('SELECT * FROM Person.Person', adventureworks_conn)
df_person_address = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
df_business_entity_address = pd.read_sql_query('SELECT * FROM Person.BusinessEntityAddress', adventureworks_conn)
df_person_phone = pd.read_sql_query('SELECT * FROM Person.PersonPhone', adventureworks_conn)

merged_df_aw = pd.merge(df_person, df_business_entity_address, on='BusinessEntityID', how='inner')
merged_df_aw = pd.merge(merged_df_aw, df_person_address, on='AddressID', how='inner')
merged_df_aw = merged_df_aw.drop(columns=['rowguid_x', 'SpatialLocation', 'ModifiedDate_x', 'rowguid_y', 'ModifiedDate_y', 'rowguid', 'ModifiedDate', 'PersonType', 'NameStyle', 'Title', 'Suffix', 'EmailPromotion', 'AdditionalContactInfo', 'Demographics', 'rowguid', 'ModifiedDate'])
merged_df_aw = pd.merge(merged_df_aw, df_person_phone, on='BusinessEntityID', how='inner')
merged_df_aw = merged_df_aw.drop(columns=['FirstName', 'MiddleName', 'LastName', 'AddressID', 'AddressTypeID', 'AddressLine2', 'StateProvinceID', 'PhoneNumberTypeID', 'ModifiedDate'])
# remove columns that are not needed

merged_df_aw


C:\Users\sheng\AppData\Local\Temp\ipykernel_41056\4245761193.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_person = pd.read_sql_query('SELECT * FROM Person.Person', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_41056\4245761193.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_person_address = pd.read_sql_query('SELECT * FROM Person.Address', adventureworks_conn)
C:\Users\sheng\AppData\Local\Temp\ipykernel_41056\4245761193.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_business_entit

,BusinessEntityID,AddressLine1,City,PostalCode,PhoneNumber
0,1,4350 Minute Dr.,Newport Hills,98006,697-555-0142
1,2,7559 Worth Ct.,Renton,98055,819-555-0175
2,3,2137 Birchwood Dr,Redmond,98052,212-555-0187
3,4,5678 Lakeview Blvd.,Minneapolis,55402,612-555-0100
4,5,9435 Breck Court,Bellevue,98004,849-555-0139
...,...,...,...,...,...
18793,20773,988 Mt. Everest Court,W. York,BD1 4SJ,1 (11) 500 555-0171
18794,20774,7413 Alpine Drive,Torrance,90505,910-555-0166
18795,20775,4764 East Avenue,Bremerton,98312,813-555-0148
18796,20776,"34334, rue Jean Mermoz",Versailles,78000,1 (11) 500 555-0171


aenc customer

In [23]:
df_aenc_customer = pd.read_sql_query('SELECT * FROM customer', aenc_conn)
df_aenc_customer

C:\Users\sheng\AppData\Local\Temp\ipykernel_41056\3911038392.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_aenc_customer = pd.read_sql_query('SELECT * FROM customer', aenc_conn)


,id,fname,lname,address,city,state,zip,phone,company_name
0,101,Michaels,Devlin,3114 Pioneer Avenue,Rutherford,NJ,07070,2015558966,The Power Group
1,102,Beth,Reiser,1033 Whippany Road,New York,NY,10154,2125558725,AMF Corp.
2,103,Erin,Niedringhaus,1990 Windsor Street,Paoli,PA,19301,2155556513,Darling Associates
3,104,Meghan,Mason,550 Dundas Street East,Knoxville,TN,37919,6155555463,P.S.C.
4,105,Laura,McCarthy,1210 Highway 36,Carmel,IN,46032,3175558437,Amo & Sons
...,...,...,...,...,...,...,...,...,...
121,552,Janice,O'Toole,654 West Hill,Nashville,TN,37320,6155553689,Greensleeves
122,553,Stevie,Nickolas,77 Recordings Circle,Tacoma,WA,96521,5095551695,It's a Hit!
123,555,Philipe,Fernandez,99 Main Street,Los Angeles,CA,90205,2135554457,Quaker Fashions
124,661,Jennifer,Stutzman,3 Back Pages Lane,Missola,IL,60505,7085556857,Stutzman Advertising


NorthWind customers

In [24]:
df_northwind_customers = pd.read_sql_query('SELECT * FROM customers', northwind_conn)

C:\Users\sheng\AppData\Local\Temp\ipykernel_41056\2771955829.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_northwind_customers = pd.read_sql_query('SELECT * FROM customers', northwind_conn)


Specifieke Data eruit halen

In [25]:
# Select specific columns and rename them
df_aw_customers_selected = merged_df_aw[['BusinessEntityID', 'AddressLine1', 'City', 'PostalCode', 'PhoneNumber']]
df_northwind_customers_selected = df_northwind_customers[['CustomerID', 'CompanyName', 'Address', 'City', 'PostalCode', 'Phone']]
df_aenc_customer_selected = df_aenc_customer[['id', 'address', 'city', 'zip', 'phone', 'company_name']]

df_aw_customers_selected['Source'] = "AW"
df_northwind_customers_selected['Source'] = "NW"
df_aenc_customer_selected['Source'] = "AC"



C:\Users\sheng\AppData\Local\Temp\ipykernel_41056\2429267359.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_northwind_customers_selected['Source'] = "NW"
C:\Users\sheng\AppData\Local\Temp\ipykernel_41056\2429267359.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aenc_customer_selected['Source'] = "AC"


Samenvoegen in Customer DF

In [26]:
# Concatenate the dataframes along axis 0 to stack them vertically
customer = pd.concat([df_northwind_customers_selected, df_aenc_customer_selected, df_aw_customers_selected], axis=0)


# BusinessEntityID has .0 at the end of the values, remove it if value is null leave it as it is
customer['BusinessEntityID'] = customer['BusinessEntityID'].apply(lambda x: str(x).replace('.0', '') if not pd.isnull(x) else x)

# Reassign values for columns as per the specified criteria
customer['customer_id'] = customer.apply(lambda row: str(row['BusinessEntityID']) if not pd.isnull(row['BusinessEntityID']) else str(row['CustomerID']) if not pd.isnull(row['CustomerID']) else row['id'], axis=1)
customer['customer_company_name'] = customer['company_name'].combine_first(customer['CompanyName'])
customer['customer_address'] = customer['address'].combine_first(customer['Address']).combine_first(customer['AddressLine1'])
customer['customer_city'] = customer['city'].combine_first(customer['City'])
customer['customer_zip'] = customer['PostalCode'].combine_first(customer['zip'])
customer['customer_phone'] = customer['phone'].combine_first(customer['PhoneNumber']).combine_first(customer['Phone'])
customer['source'] = customer['Source']

# Drop redundant columns
customer.drop(['BusinessEntityID', 'CustomerID', 'CompanyName', 'Address', 'AddressLine1', 'City', 'PostalCode', 'PhoneNumber', 'Phone'], axis=1, inplace=True)

# Reorder the columns
customer = customer[['customer_id', 'customer_company_name', 'customer_address', 'customer_city', 'customer_zip', 'customer_phone','source']]

# Display where id is 101
customer=customer.where(pd.notna(customer), None)
customer

,customer_id,customer_company_name,customer_address,customer_city,customer_zip,customer_phone,source
0,ALFKI,Alfreds Futterkiste,Obere Str. 57,Berlin,12209,030-0074321,NW
1,ANATR,Ana Trujillo Emparedados y helados,Avda. de la Constitución 2222,México D.F.,05021,(5) 555-4729,NW
2,ANTON,Antonio Moreno Taquería,Mataderos 2312,México D.F.,05023,(5) 555-3932,NW
3,AROUT,Around the Horn,120 Hanover Sq.,London,WA1 1DP,(171) 555-7788,NW
4,BERGS,Berglunds snabbk?p,Berguvsv?gen 8,Lule?,S-958 22,0921-12 34 65,NW
...,...,...,...,...,...,...,...
18793,20773,None,988 Mt. Everest Court,W. York,BD1 4SJ,1 (11) 500 555-0171,AW
18794,20774,None,7413 Alpine Drive,Torrance,90505,910-555-0166,AW
18795,20775,None,4764 East Avenue,Bremerton,98312,813-555-0148,AW
18796,20776,None,"34334, rue Jean Mermoz",Versailles,78000,1 (11) 500 555-0171,AW


Convert types to varchar

In [27]:

customer.dtypes

customer_id              object
customer_company_name    object
customer_address         object
customer_city            object
customer_zip             object
customer_phone           object
source                   object
dtype: object

In [28]:
united_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=UnitedOutdoors; Trusted_Connection=yes')
unitedCursor = united_conn.cursor()
unitedCursor

In [29]:
for index, row in customer.iterrows():
    try:
        query = """INSERT INTO customer (
            customer_id,
            customer_company_name, 
            customer_address, 
            customer_city,
            customer_zip,
            customer_phone,
            customer_source
        ) VALUES (?, ?, ?, ?, ?, ?, ?)"""  
        
        values = (
            row['customer_id'],
            row['customer_company_name'],
            row['customer_address'],
            row['customer_city'],
            row['customer_zip'],
            row['customer_phone'],
            row['source']
        )

        unitedCursor.execute(query, values)

    except pyodbc.IntegrityError as e:
        continue
    except pyodbc.Error as e:
        print("Er is een fout opgetreden: ", e)
        print("Foutieve query: ", query)
    
unitedCursor.commit()

print("Data is succesvol geïmporteerd in de UnitedOutdoors database")

Data is succesvol geïmporteerd in de UnitedOutdoors database


In [30]:
aenc_conn.close()